In [1]:
lan = ["eng", "esp", "amh","arq","ary","hau","kin","mar","tel"]
# lan="eng"
#sentence-transformers/distiluse-base-multilingual-cased-v2

In [2]:
num_labels=1
seed=7
batch_size=16
lr=2e-5
# set the number of epochs
n_epochs = 15 #3
l_lr=5e-2
emded_d = 1024
max_len=92
early_stoping_patience=3


In [3]:
import random

random.seed(seed)


In [4]:
import re
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [5]:
from transformers import  XLMRobertaTokenizer, XLMRobertaModel, XLMRobertaForSequenceClassification


In [6]:

# -*- coding: utf-8 -*-


"""# Import the Libraries"""
import pandas as pd
from collections import Counter
import sys
import json
import json
# visualization libraries
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# pytorch libraries
import torch # the main pytorch library
import torch.nn as nn # the sub-library containing Softmax, Module and other useful functions
import torch.optim as optim # the sub-library containing the common optimizers (SGD, Adam, etc.)

# huggingface's transformers library
from transformers import RobertaForSequenceClassification, AutoTokenizer

# huggingface's datasets library
from datasets import load_dataset
from datasets import  load_from_disk

# the tqdm library used to show the iteration progress
import tqdm
from tqdm import tqdm
# tqdmn = tqdm.notebook.tqdm#tqdm.tqdm# tqdm.notebook.tqdm

from datasets import load_dataset
import os
from transformers import get_scheduler
import torch
from datasets import Dataset
from datasets import load_from_disk
import sys
from sklearn.metrics import confusion_matrix
import copy

In [7]:
# train Load the File
df_final=pd.DataFrame()
df_final['PairID']=None
df_final['Text']=None

for i, lang in enumerate(lan):
    df_str_rel_train = pd.read_csv("Track A/"+lang+"/"+lang+"_train.csv")
    df_final=pd.concat([df_str_rel_train, df_final])

In [8]:
df_final

,PairID,Text,Score
0,TEL-train-00001,"""జమ్మూకాశ్మీర్లో మంచుకొండచరియలు విరిగిపడి గురు...",0.88
1,TEL-train-00002,"""కూతురుపైనే కన్నతండ్రి అత్యాచారానికి పాల్పడిన ...",0.87
2,TEL-train-00003,"""గంగా నదిని కాపాడేందుకు ప్రత్యేక చట్టం తీసుకుర...",0.87
3,TEL-train-00004,"""కరోనా వైరస్పై అమెరికా ప్రభుత్వానికి వ్యతిరేకం...",0.84
4,TEL-train-00005,"""కరోనా వైరస్ ( కొవిడ్ - 19 ) మనుషులపైనే కాదు జ...",0.84
...,...,...,...
5495,ENG-train-5495,A young boy pounding on an anvil.\nWoman sits ...,0.00
5496,ENG-train-5496,I love how he recognized his wife tempered his...,0.00
5497,ENG-train-5497,I actually read a chapter or two beyond that p...,0.00
5498,ENG-train-5498,A boy gives being in the snow two thumbs up.\n...,0.00


In [9]:
df_final["PairID"]

0       TEL-train-00001
1       TEL-train-00002
2       TEL-train-00003
3       TEL-train-00004
4       TEL-train-00005
             ...       
5495     ENG-train-5495
5496     ENG-train-5496
5497     ENG-train-5497
5498     ENG-train-5498
5499     ENG-train-5499
Name: PairID, Length: 15123, dtype: object

In [10]:
# train Load the File
df_str_rel_dev={}
# df_final=pd.DataFrame()
for i, lang in enumerate(lan):
    df_str_rel_dev[lang]= pd.read_csv("Track A/"+lang+"/"+lang+"_dev.csv")

In [11]:
df_str_rel_dev['eng']

,PairID,Text
0,ENG-dev-0000,The story is gripping and interesting.\nIt's a...
1,ENG-dev-0001,The majority of Southeast Alaska 's area is pa...
2,ENG-dev-0002,and from your post i think you are to young to...
3,ENG-dev-0003,The film 's success also made Dreamworks Anima...
4,ENG-dev-0004,I am still confused about how I feel about thi...
...,...,...
245,ENG-dev-0245,thats just how they are :( its a shame lol\nIt...
246,ENG-dev-0246,I feel sorry for the books that I will read af...
247,ENG-dev-0247,Uwe Seeler -LRB- born 5 November 1936 in Hambu...
248,ENG-dev-0248,Waco is a city in and the county seat of McLen...


In [12]:
df_final

,PairID,Text,Score
0,TEL-train-00001,"""జమ్మూకాశ్మీర్లో మంచుకొండచరియలు విరిగిపడి గురు...",0.88
1,TEL-train-00002,"""కూతురుపైనే కన్నతండ్రి అత్యాచారానికి పాల్పడిన ...",0.87
2,TEL-train-00003,"""గంగా నదిని కాపాడేందుకు ప్రత్యేక చట్టం తీసుకుర...",0.87
3,TEL-train-00004,"""కరోనా వైరస్పై అమెరికా ప్రభుత్వానికి వ్యతిరేకం...",0.84
4,TEL-train-00005,"""కరోనా వైరస్ ( కొవిడ్ - 19 ) మనుషులపైనే కాదు జ...",0.84
...,...,...,...
5495,ENG-train-5495,A young boy pounding on an anvil.\nWoman sits ...,0.00
5496,ENG-train-5496,I love how he recognized his wife tempered his...,0.00
5497,ENG-train-5497,I actually read a chapter or two beyond that p...,0.00
5498,ENG-train-5498,A boy gives being in the snow two thumbs up.\n...,0.00


In [13]:
# Creating a column "Split_Text" which is a list of two sentences.

df_final['Split_Text'] = df_final['Text'].apply(lambda x: x.split("\n"))
df_final.head()

,PairID,Text,Score,Split_Text
0,TEL-train-00001,"""జమ్మూకాశ్మీర్లో మంచుకొండచరియలు విరిగిపడి గురు...",0.88,"[""జమ్మూకాశ్మీర్లో మంచుకొండచరియలు విరిగిపడి గుర..."
1,TEL-train-00002,"""కూతురుపైనే కన్నతండ్రి అత్యాచారానికి పాల్పడిన ...",0.87,"[""కూతురుపైనే కన్నతండ్రి అత్యాచారానికి పాల్పడిన..."
2,TEL-train-00003,"""గంగా నదిని కాపాడేందుకు ప్రత్యేక చట్టం తీసుకుర...",0.87,"[""గంగా నదిని కాపాడేందుకు ప్రత్యేక చట్టం తీసుకు..."
3,TEL-train-00004,"""కరోనా వైరస్పై అమెరికా ప్రభుత్వానికి వ్యతిరేకం...",0.84,"[""కరోనా వైరస్పై అమెరికా ప్రభుత్వానికి వ్యతిరేక..."
4,TEL-train-00005,"""కరోనా వైరస్ ( కొవిడ్ - 19 ) మనుషులపైనే కాదు జ...",0.84,"[""కరోనా వైరస్ ( కొవిడ్ - 19 ) మనుషులపైనే కాదు ..."


In [14]:
df_final.loc[0]["Split_Text"]

0    ["జమ్మూకాశ్మీర్లో మంచుకొండచరియలు విరిగిపడి గుర...
0    ["गुरु नानक देव, त्याचे शिकवण आणि आशीर्वाद आता...
0    [Izi serivisi benshi bigaragara ko batarasoban...
0    [Haka ya furta a cikin jawabin sa na murnar ci...
0    [تنفرد بنشرها “كود”: سلسلة “كيفاش السلاطين ديا...
0    [يعطيك الصحة ام وليد جربتها ذركا راني ناكل فيه...
0    [መግለጫውን የተከታተለው የአዲስ አበባው ዘጋቢያችን ሰሎሞን ሙጬ ዝርዝር ...
0    [Una mujer a punto de comer pescado., Una muje...
0    [It that happens, just pull the plug., if that...
Name: Split_Text, dtype: object

# Generate submission file
```
# This is formatted as code
```



### Append prediction to dataframe

> Indented block

In [15]:
train_pdf=df_final.sample(frac=0.9,random_state=200)
dev_pdf=df_final.drop(train_pdf.index)

test_pdf = df_str_rel_dev

In [16]:
train_pdf

,PairID,Text,Score,Split_Text
1241,ESP-train-1241,"Rodrigo Cortés, que parece encantado de habers...",0.29,"[Rodrigo Cortés, que parece encantado de haber..."
1342,ENG-train-1342,You cannot win domestic wars (even non-violent...,0.66,[You cannot win domestic wars (even non-violen...
231,ENG-train-0231,Give Palmela and her five sisters a phone call...,0.91,[Give Palmela and her five sisters a phone cal...
418,ENG-train-0418,"It is because of the way the man says ""S"". It ...",0.84,"[It is because of the way the man says ""S"". It..."
756,kin_train_00757,"Nubwo byagabanyutse, inzobere mu buzima zagiye...",0.62,"[Nubwo byagabanyutse, inzobere mu buzima zagiy..."
...,...,...,...,...
132,TEL-train-00133,"""నిఖిల్ తాజా చిత్రంగా ' ముద్ర ' రూపొందుతోంది ....",0.70,"[""నిఖిల్ తాజా చిత్రంగా ' ముద్ర ' రూపొందుతోంది ..."
61,kin_train_00062,Muri 1979 yahawe igihembo cy'amahoro kitiriwe ...,0.75,[Muri 1979 yahawe igihembo cy'amahoro kitiriwe...
439,ESP-train-0439,"Los seres humanos evolucionaron ""de una especi...",0.38,"[Los seres humanos evolucionaron ""de una espec..."
806,ARQ-train-0807,والله ضحكتيني خلاه\n ما تقرايش اليوم,0.16,"[والله ضحكتيني خلاه, ما تقرايش اليوم]"


In [17]:
dev_pdf

,PairID,Text,Score,Split_Text
1688,hau_train_01689,"Shugaban Ƙungiyar Dattawan Yankin Middle Belt,...",0.17,[Shugaban Ƙungiyar Dattawan Yankin Middle Belt...
1688,ENG-train-1688,"It was played in a variety of forms , with 52 ...",0.62,"[It was played in a variety of forms , with 52..."
1736,ENG-train-1736,"From there, things slowly get inexplicable wit...",0.62,"[From there, things slowly get inexplicable wi..."
1745,ENG-train-1745,The 1890 Atlantic hurricane season ran through...,0.62,[The 1890 Atlantic hurricane season ran throug...
1752,ENG-train-1752,Jhangi is one of the 51 union councils of Abbo...,0.62,[Jhangi is one of the 51 union councils of Abb...
...,...,...,...,...
5421,ENG-train-5421,"Based on my personal experience, I would say t...",0.09,"[Based on my personal experience, I would say ..."
5425,ENG-train-5425,"Thalia was a woman who didn't need a big, stro...",0.09,"[Thalia was a woman who didn't need a big, str..."
5436,ENG-train-5436,Perhaps you can start something in the concess...,0.06,[Perhaps you can start something in the conces...
5470,ENG-train-5470,This appears to be a Muslim woman with her leg...,0.06,[This appears to be a Muslim woman with her le...


In [18]:
valid_true_labels= dev_pdf["Score"]

In [41]:
test_pdf

{'eng':            PairID                                               Text
 0    ENG-dev-0000  The story is gripping and interesting.\nIt's a...
 1    ENG-dev-0001  The majority of Southeast Alaska 's area is pa...
 2    ENG-dev-0002  and from your post i think you are to young to...
 3    ENG-dev-0003  The film 's success also made Dreamworks Anima...
 4    ENG-dev-0004  I am still confused about how I feel about thi...
 ..            ...                                                ...
 245  ENG-dev-0245  thats just how they are :( its a shame lol\nIt...
 246  ENG-dev-0246  I feel sorry for the books that I will read af...
 247  ENG-dev-0247  Uwe Seeler -LRB- born 5 November 1936 in Hambu...
 248  ENG-dev-0248  Waco is a city in and the county seat of McLen...
 249  ENG-dev-0249  religious extremism continues in pakistan desp...
 
 [250 rows x 2 columns],
 'esp':            PairID                                               Text
 0    ESP-dev-0000  Notable es la carta de Robert

In [19]:
train_dataset =  Dataset.from_pandas(train_pdf)
train_dataset = train_dataset.rename_column("PairID", "sentence_id")
train_dataset = train_dataset.rename_column("Text", "words")


valid_dataset = Dataset.from_pandas(dev_pdf)
valid_dataset = valid_dataset.rename_column("PairID", "sentence_id")
valid_dataset = valid_dataset.rename_column("Text", "words")



In [20]:
test_dataset={}
for lang in lan:
    test_dataset[lang] =  Dataset.from_pandas(test_pdf[lang])
    test_dataset[lang] = test_dataset[lang].rename_column("PairID", "sentence_id")
    test_dataset[lang] = test_dataset[lang].rename_column("Text", "words")
    test_dataset

In [21]:
test_dataset

{'eng': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 250
 }),
 'esp': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 140
 }),
 'amh': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 95
 }),
 'arq': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 97
 }),
 'ary': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 71
 }),
 'hau': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 212
 }),
 'kin': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 102
 }),
 'mar': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 293
 }),
 'tel': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 130
 })}

In [22]:
train_dataset

Dataset({
    features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
    num_rows: 13611
})

In [23]:
def sentence_len_distribution(sentences):
    sentences_len=[]

    for sentence in sentences:
        sentences_len.append(len(sentence))
    return Counter(sentences_len)

In [24]:
test_dataset['eng']["words"][0]

"The story is gripping and interesting.\nIt's a brilliant, compelling, and heartfelt story."

In [25]:
train_dataset["Split_Text"]

[['Rodrigo Cortés, que parece encantado de haberse conocido, nos marea y despista con un estilo visual variado pero no excesivamente original.',
  'Una de esas películas que parece que lleva toda la vida conviviendo y que a uno le cuesta trabajo sentarse a visionar.'],
 ['You cannot win domestic wars (even non-violent ones) through legal processes alone.',
  'internal war (even non-violent) cannot be won only with the help of law and law.'],
 ['Give Palmela and her five sisters a phone call.',
  'Call up Palmela and her five sisters.'],
 ['It is because of the way the man says "S". It makes me uncomfortable.',
  "it's because of how the guy says his ss....uggghhh, it makes me cringe."],
 ['Nubwo byagabanyutse, inzobere mu buzima zagiye zigaragaza ko gukora imibonano mpuzabitsina bifite inyungu zirimo kugabanya imihangayiko, gukora neza k’umutima no gusinzira neza.',
  'Abashakashatsi bagaragaza ko gukora imibonano mpuzabitsina bigabanya umunaniro kandi bigatuma umuntu abasha gusinzira 

In [26]:
# !pip install -U sentence-transformers

# from sentence_transformers import SentenceTransformer, util

In [27]:
# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


In [28]:

# sentences = ["I'm happy", "I'm full of happiness"]


# #Compute embedding for both lists
# embedding_1= model.encode(sentences[0], convert_to_tensor=True)
# embedding_2 = model.encode(sentences[1], convert_to_tensor=True)

# util.pytorch_cos_sim(embedding_1, embedding_2)


Train data


In [29]:
# true_scores = train_pdf['Score'].values
# pred_scores = []

# for index,row in train_pdf.iterrows():
#   #Compute embedding for both lists
#   embedding_1= model.encode(row["Split_Text"][0], convert_to_tensor=True)
#   embedding_2 = model.encode(row["Split_Text"][1], convert_to_tensor=True)


#   # Overlap score
#   pred_scores.append(util.pytorch_cos_sim(embedding_1, embedding_2))

In [30]:
# pred=[]
# for i in range(len(pred_scores)):
#   pred.append(float(pred_scores[i][0]))
# pred_scores=pred

In [31]:
# for i in range(len(pred_scores)):
#   pred_scores[i] =float(pred_scores[i])

test data

In [34]:
train_dataset

Dataset({
    features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
    num_rows: 13611
})

In [35]:
train_pdf['Split_Text'][1]

1    [خالتي خيرة وليتي تعرفي السلاح,  كون جيتو تحوس...
1    ["२०१ च्या तुलनेत आंतरराष्ट्रीय संस्था आणि एजन...
1    [RASHIN TSARO: ‘Yan bindiga sun arce da Kwamis...
1    [Avuga ko agahinda ke katangiye gukura ubwo ya...
1    [Las protestas volvieron a Honduras, en este c...
1    ["కూతురుపైనే కన్నతండ్రి అత్యాచారానికి పాల్పడిన...
1    [الوضعية الوبائية لفيروس كورونا فهاد 24 ساعة: ...
1    [A black dog running through water., A black d...
1    [የኛ ዴሞክራሲ የእንግሊዝ ስርዓት ነው ፤ ለጊዜው ሊያገለግል ይችላል ።\...
Name: Split_Text, dtype: object

In [36]:
# test_pdf[['PairID', 'Pred_Score']].to_csv('pred_eng.csv', index=False)

In [37]:
roberta_version = 'xlm-roberta-large'#"/home/naive123/nlp/Sumit/models_repo/muril_large_model"#
roberta_version_tokenizer = 'xlm-roberta-large'#"/home/naive123/nlp/Sumit/models_repo/muril_large_tokenizer"
tokenizer = AutoTokenizer.from_pretrained(roberta_version_tokenizer)
tokenizer.model_max_length = max_len
print(tokenizer.model_max_length )

92


In [38]:
#Cuda device


not_equal=0
word_ids_of_sentences = []
repeat_batch=0


valid_tokenization = []
npt_equql_flag = 0
def add_encodings(example):
    global npt_equql_flag
    global word_ids_of_sentences
    global repeat_batch


    example_words=  example['words']   # for bert type models

    # atoc=tokenizer.convert_ids_to_tokens(tokenizer(example['words'],is_split_into_words=True)["input_ids"],skip_special_tokens=False)[1:-1]
    # print(example)
    # if len(example['Split_Text'])!=2:
    #     example['Split_Text']=["not equal", "not equal"]
    #     not_equal+=1
    # print(example)
    print(example["sentence_id"])
    # try:
    #     if example["sentence_id"][8:11]=='amh':
    #         example['Split_Text']=example['Split_Text'].split("\t")
    #         print(example['Split_Text'])
    #         encodings = tokenizer(example['Split_Text'][0]+"</s>"+example['Split_Text'][1], truncation=True, padding='max_length', is_split_into_words=False)
    #     else:
    #         encodings = tokenizer(example['Split_Text'][0]+"</s>"+example['Split_Text'][1], truncation=True, padding='max_length', is_split_into_words=False)
    #     #print(encodings)
    # except:
    #     print('error')
    #     print(example)
    if example["sentence_id"][8:11]=='amh':
        print(example['Split_Text'])
        example['Split_Text']=example['Split_Text'][0].split("\t")
        print(example['Split_Text'])
        encodings = tokenizer(example['Split_Text'][0]+"</s>"+example['Split_Text'][1], truncation=True, padding='max_length', is_split_into_words=False)
    else:
        encodings = tokenizer(example['Split_Text'][0]+"</s>"+example['Split_Text'][1], truncation=True, padding='max_length', is_split_into_words=False)
    #print(encodings)
    if "Score" in example.keys():
      score=example['Score']
    else:
      score=0

    # return the encodings and the extended ner_tags
    return { **encodings, 'labels': score}


In [39]:
print("tokenizer.model_max_length=",tokenizer.model_max_length)

print("encodinmmg start")
#valid_dataset = valid_dataset.map(add_encodings)

# print("encodinmmg complete")
#aaa=add_encodings(train_dataset[1166])
#aaa=add_encodings(train_dataset[1167])

print("encodinmmg start")

print(train_dataset.num_rows)
train_dataset = train_dataset.map(add_encodings,load_from_cache_file=False, batch_size =10000)#load_from_cache_file=False,
#train_subwordid_1s, train_subwordid_2s, train_subwordid_3s = create_Subwordid(word_ids_for_curr_sentence)
print(train_dataset[0])
print(train_dataset[1])
print(train_dataset[2])
print("encodinmmg complete")

tokenizer.model_max_length= 92
encodinmmg start
encodinmmg start
13611


Map:   0%|          | 0/13611 [00:00<?, ? examples/s]

ESP-train-1241
ENG-train-1342
ENG-train-0231
ENG-train-0418
kin_train_00757
ARY-train-0425
hau_train_00613
hau_train_00751
Pair_ID_amh_train_650
['የካቲት 5 ቀን 2009 ዓም ።\tሐምሌ 14 ቀን 2009 ዓም ።']
['የካቲት 5 ቀን 2009 ዓም ።', 'ሐምሌ 14 ቀን 2009 ዓም ።']
ENG-train-1132
ENG-train-1113
ENG-train-5277
MAR-train-00539
MAR-train-01078
ENG-train-3075
TEL-train-00288
ENG-train-2346
ENG-train-1654
hau_train_01664
kin_train_00071
ESP-train-0782
hau_train_00790
ESP-train-0225
Pair_ID_amh_train_594
['የህዝብ ተወካዮች ምክር ቤት 5ኛ ዓመት የስራ ዘመን 10ኛ መደበኛ ስብሰባ በዛሬው ዕለት ተካሂዷል ።\tበረቂቁ ዙሪያ በዛሬው ዕለት በህዝብ ተወካዮች ምክር ቤት የህዝብ ውይይት አካሂዷል ፡፡']
['የህዝብ ተወካዮች ምክር ቤት 5ኛ ዓመት የስራ ዘመን 10ኛ መደበኛ ስብሰባ በዛሬው ዕለት ተካሂዷል ።', 'በረቂቁ ዙሪያ በዛሬው ዕለት በህዝብ ተወካዮች ምክር ቤት የህዝብ ውይይት አካሂዷል ፡፡']
TEL-train-00446
TEL-train-00964
hau_train_00387
ENG-train-5472
ENG-train-3898
hau_train_00658
ENG-train-0587
ARQ-train-0769
hau_train_00923
Pair_ID_amh_train_623
['ይህ ገጽ መጨረሻ የተቀየረው እ.ኣ.አ በ18 :55 ፣ 29 ጁላይ 2013 ዓ.ም. ነበር ።\tይህ ገጽ መጨረሻ የተቀየረው እ.ኣ.አ በ19 :23 ፣ 1 ጁላይ 2009 ዓ.ም. ነበር

In [40]:
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask','labels'])


In [41]:
print("encodinmmg start")
valid_true_labels = valid_dataset["Score"]
print(valid_dataset.num_rows)
valid_dataset = valid_dataset.map(add_encodings,load_from_cache_file=False, batch_size =10000)#load_from_cache_file=False,

encodinmmg start
382


Map:   0%|          | 0/382 [00:00<?, ? examples/s]

hau_train_01689
ENG-train-1688
ENG-train-1736
ENG-train-1745
ENG-train-1752
ENG-train-1773
ENG-train-1776
ENG-train-1779
ENG-train-1801
ENG-train-1804
ENG-train-1822
ENG-train-1844
ENG-train-1847
ENG-train-1861
ENG-train-1863
ENG-train-1900
ENG-train-1929
ENG-train-1938
ENG-train-1940
ENG-train-1947
ENG-train-1950
ENG-train-1952
ENG-train-1953
ENG-train-1954
ENG-train-1955
ENG-train-1959
ENG-train-1964
ENG-train-1978
ENG-train-1991
ENG-train-1992
ENG-train-2002
ENG-train-2006
ENG-train-2019
ENG-train-2033
ENG-train-2068
ENG-train-2071
ENG-train-2081
ENG-train-2090
ENG-train-2117
ENG-train-2118
ENG-train-2127
ENG-train-2130
ENG-train-2139
ENG-train-2171
ENG-train-2174
ENG-train-2176
ENG-train-2199
ENG-train-2210
ENG-train-2226
ENG-train-2227
ENG-train-2229
ENG-train-2252
ENG-train-2256
ENG-train-2265
ENG-train-2272
ENG-train-2277
ENG-train-2292
ENG-train-2305
ENG-train-2308
ENG-train-2310
ENG-train-2320
ENG-train-2326
ENG-train-2340
ENG-train-2343
ENG-train-2378
ENG-train-2383
ENG-train

In [42]:
valid_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask','labels'])


In [43]:
valid_dataset

Dataset({
    features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 382
})

In [44]:
# print("encodinmmg start")
# print(test_dataset.num_rows)

# # test_labels = test_dataset['labels']
# test_dataset = test_dataset.map(add_encodings, batch_size =10000, load_from_cache_file=False)



In [45]:
test_dataset

{'eng': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 250
 }),
 'esp': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 140
 }),
 'amh': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 95
 }),
 'arq': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 97
 }),
 'ary': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 71
 }),
 'hau': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 212
 }),
 'kin': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 102
 }),
 'mar': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 293
 }),
 'tel': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 130
 })}

In [47]:
# test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


In [48]:
test_dataset

{'eng': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 250
 }),
 'esp': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 140
 }),
 'amh': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 95
 }),
 'arq': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 97
 }),
 'ary': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 71
 }),
 'hau': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 212
 }),
 'kin': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 102
 }),
 'mar': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 293
 }),
 'tel': Dataset({
     features: ['sentence_id', 'words'],
     num_rows: 130
 })}

In [ ]:
# test_data = torch.utils.data.DataLoader( test_dataset, batch_size=batch_size )


In [49]:
torch.manual_seed(seed)
# initialize the model and provide the 'num_labels' used to create the classification layer
model = XLMRobertaForSequenceClassification.from_pretrained(roberta_version,num_labels=1)

print("model loded")

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model loded


In [50]:
device =torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)


cuda


In [51]:
torch.cuda.is_available()


True

In [52]:
torch.__version__

'2.0.1+cu117'

In [53]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [135]:

!nvidia-smi


Wed Jan 17 20:09:46 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     Off  | 00000000:81:00.0 Off |                  Off |
| 33%   28C    P8     6W / 260W |   5560MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [54]:
# set the model in 'train' mode and send it to the device
model.train().to(device)

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_fe

In [55]:
model.config.num_labels

1

In [56]:
model.num_labels

1

In [57]:

optimizer = optim.AdamW([ { 'params':model.parameters(), 'lr':lr}])#,{'params':weight_layer, 'lr':l_lr},{'params':bias_layer, 'lr':l_lr}, {'params':crf_model.parameters(), 'lr':l_lr




# batch the train data so that each batch contains 4 examples (using 'batch_size')
train_data = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
valid_data = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size )


num_training_steps = (n_epochs * len(train_data))/16
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps)


train_loss = []





In [59]:
import mlflow
mlflow.set_tracking_uri("http://172.17.25.123:8080/")

In [ ]:
# import mlflow
# mlflow.set_tracking_uri("http://172.17.25.123:8080/")
import tqdm
tqdmn = tqdm.notebook.tqdm
print("training start")
previous_total_valid_loss = sys.float_info.max
valid_min_loss = previous_total_valid_loss
early_stoping_flag=0
# p_directory = "/content/drive/MyDrive/textual_relatedness/models/xlm_64_1e_4seed11/"
hyperparameter_combinations = [
    {'seed': 11, 'batch_size': 64, 'lr': 2e-4, 'n_epochs': 5, 'max_len': 64, 'early_stopping_patience': 3},
    {'seed': 7, 'batch_size': 64, 'lr': 1e-4, 'n_epochs': 5,  'max_len': 64, 'early_stopping_patience': 3},
    {'seed': 15, 'batch_size': 64, 'lr': 8e-5, 'n_epochs': 5, 'max_len': 64, 'early_stopping_patience': 3},
    {'seed': 3, 'batch_size': 64, 'lr': 1e-5, 'n_epochs': 5, 'max_len': 64, 'early_stopping_patience': 3},
    {'seed': 1, 'batch_size': 64, 'lr': 2e-5, 'n_epochs': 5,  'max_len': 64, 'early_stopping_patience': 3},
    {'seed': 5, 'batch_size': 64, 'lr': 5e-5, 'n_epochs': 5, 'max_len': 64, 'early_stopping_patience': 3},
]

for version, hyperparams in enumerate(hyperparameter_combinations):
    p_directory = f"multiple_models_/XLM_roberta{hyperparameter_combinations[version]['batch_size']}_{hyperparameter_combinations[version]['lr']}seed{hyperparameter_combinations[version]['seed']}/"
# Start MLflow run
    # model = AutoModelForSequenceClassification.from_pretrained(roberta_version,num_labels=1)
    # model.train().to(device)
    with mlflow.start_run(run_name=f"version_XLM_Roberta{version}") as run:
        for param, value in hyperparams.items():
            mlflow.log_param(param, value)
        seed
        torch.manual_seed(seed)
        model = XLMRobertaForSequenceClassification.from_pretrained(roberta_version,num_labels=1,ignore_mismatched_sizes=True)
        model.train().to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=hyperparams['lr'])
        num_training_steps = (n_epochs * len(train_data))/16
        lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps)
        # lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)  # You can adjust the scheduler as needed
        # iterate through the data 'n_epochs' times
        train_accuracy_list = []
        valid_accuracy_list = []
        error=0
        for epoch in tqdmn(range(n_epochs)):
            count=0
            print("epoch", epoch)
            directory=p_directory+"epoch_"+str(epoch)
            print("\n")
            print(directory)
            #model = BertForTokenClassification.from_pretrained(directory)
            #model.train().to(device)
            current_loss = 0
            total_loss=0
            # iterate through each batch of t    he train data
            k=0
            model.train().to(device)          
            train_predictions=[]
            for i, batch in enumerate(tqdmn(train_data)):
                batch = { k: v.to(device) for k, v in batch.items()}
                # print(batch['input_ids'])
                count+=1
                outputs=model(**batch)
                # try:
                #     outputs = model(**batch)
                # except:
                #     error=error+1
                #     continue
                loss =outputs.loss
                logits=outputs.logits
                loss.backward()
                # predictions = torch.argmax(logits, dim=1)
                predictions=logits.cpu().detach()
                # print("predictions", predictions)
                train_predictions.extend(list(predictions))
                current_loss += loss.item()
                if i % 4 == 0 and i > 0:       # #  # #       #if i % 8 == 0 and i > 0:
                    # update the model using the optimizer
                    optimizer.step()
                    lr_scheduler.step()
                    # once we update the model we set the gradients to zero
                    optimizer.zero_grad()
                    # store the loss value for visualization
                    train_loss.append(current_loss/16)
                    total_loss=total_loss + current_loss/64
                    current_loss = 0
                # update the model one l2064469413403458ast time for this epoch
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            # model.save_pretrained(directory)
            print(directory + ' files saved')
            print("error:", error)
            print('total_loss',total_loss)
            train_labels = train_pdf["Score"].values # Get your actual training labels
            # print(train_predictions)
            # print('**********')
            # print(train_labels)
            train_predictions_clean = [item[0] for item in train_predictions]
            # print("train_predictions", train_predictions_clean)
            train_final_predictions=[value.item() for value in train_predictions_clean]
            # print("train_final_predictions", train_final_predictions)
            print('*********')
            # print(train_final_predictions)
            # print(train_labels)
            # train_accuracy = round(pearsonr(train_final_predictions,train_labels)[0],2)
            # print("train_accuracy", train_accuracy)
            # train_accuracy_list.append(train_accuracy)
         # Save the model with hyperparameter details in the directory
            model_directory = os.path.join(p_directory, f"epoch_{epoch}")
            os.makedirs(model_directory, exist_ok=True)
            model.save_pretrained(model_directory)
            # mlflow.pytorch.save_model(model, model_directory)
            mlflow.log_metric("train_loss", total_loss)
            # mlflow.log_metric("train_accuracy", train_accuracy)
        #     validation####################################
            model.eval().to(device)
            valid_loss = []
            valid_current_loss = 0
            valid_total_loss=0
            # iterate through each batch of t    he train data
            valid_predictions=[]
            for i, batch in enumerate(tqdmn(valid_data)):
                #print(batch)
                batch = { k: v.to(device) for k, v in batch.items()}
                #print(batch["labels"].shape)
                #print(batch["input_ids"].shape)
                # send 'input_ids', 'attention_mask' and 'labels' to the model
                outputs = model(**batch)
# pandas==2.1.4
                # print(outputs)
                loss=outputs.loss
                logits = outputs.logits
                predictions=logits.cpu()
                # predictions = torch.argmax(logits, dim=1)
                valid_predictions.extend(predictions.detach())
                valid_current_loss += loss.item()
                #print(valid_current_loss)
                valid_loss.append(valid_current_loss/batch_size)
                valid_total_loss=valid_total_loss + valid_current_loss/batch_size
                valid_current_loss = 0
                    #print("valid_total_loss ",valid_total_loss)
            print("valid_total_loss ",valid_total_loss)
            valid_labels = dev_pdf["Score"].values  # Get your actual validation labels
            dev_predictions_clean = [item[0] for item in valid_predictions]
            dev_predictions_final=[value.item() for value in dev_predictions_clean]
            valid_accuracy =  round(pearsonr(dev_predictions_final,valid_labels)[0],2)
            print("valid_accuracy", valid_accuracy)
            valid_accuracy_list.append(valid_accuracy)
            mlflow.log_metric("valid_total_loss ",valid_total_loss)
            mlflow.log_metric("valid_accuracy", valid_accuracy)
             # Log final accuracies for the last epoch
            # mlflow.log_metric("final_train_accuracy", train_accuracy_list[-1])
            mlflow.log_metric("final_valid_accuracy", valid_accuracy_list[-1])
            # valid_loss_epoch.append(valid_total_loss)
            if valid_total_loss < valid_min_loss:
                valid_min_loss = valid_total_loss
        #         model.save_pretrained(directory+"best_e_"+str(epoch))
            if previous_total_valid_loss < valid_total_loss:
                early_stoping_flag += 1
            else:
                early_stoping_flag=0
            previous_total_valid_loss = valid_total_loss
            if early_stoping_flag == early_stoping_patience:
                print("early_stoping occure")
                break
print('All files saved')

client = MlflowClient()
best_run = client.search_runs(order_by=["metrics.valid_accuracy desc"]).iloc[0]
best_model_uri = best_run.info.artifact_uri + "/models"
print(f"The best model is stored at: {best_model_uri}")

# Command to see all the metrics on MLflow UI
print(f"Visit MLflow UI: {mlflow.get_tracking_uri()}")

training start


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/15 [00:00<?, ?it/s]

epoch 0


multiple_models_/XLM_roberta64_0.0002seed11/epoch_0


  0%|          | 0/851 [00:00<?, ?it/s]

...

  0%|          | 0/851 [00:00<?, ?it/s]

multiple_models_/XLM_roberta64_5e-05seed5/epoch_9 files saved
error: 0
total_loss 0.31470616740261903
*********


  0%|          | 0/24 [00:00<?, ?it/s]

valid_total_loss  0.021863945032237098
valid_accuracy 0.72
epoch 10


multiple_models_/XLM_roberta64_5e-05seed5/epoch_10


  0%|          | 0/851 [00:00<?, ?it/s]

In [154]:
# print("training start")
# previous_total_valid_loss = sys.float_info.max
# valid_min_loss = previous_total_valid_loss
# early_stoping_flag=0

# p_directory = "/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/"

# # iterate through the data 'n_epochs' times
# for epoch in range(n_epochs):
#     print(epoch)
#     directory=p_directory+"epoch_"+str(epoch)
#     print(directory)

#     #model = BertForTokenClassification.from_pretrained(directory)
#     #model.train().to(device)

#     current_loss = 0
#     total_loss=0
#     # iterate through each batch of t    he train data
#     k=0
#     for i, batch in enumerate(train_data):

#         # move the batch tensors to the same device as the
#         #print(batch)
#         batch = { k: v.to(device) for k, v in batch.items() }
#         # print(batch)


#         outputs = model(**batch)
#         # print(outputs)
#         # print(outputs.loss)
#         # print(outputs.logits.shape)
#         # the outputs are of shape (loss, logits)
#         loss =outputs.loss

#         loss.backward()
#         # NOTE: if we append `loss` (a tensor) we will force the GPU to save
#         # the loss into its memory, potentially filling it up. To avoid this
#         # we rather store its float value, which can be accessed through the
#         # `.item` method
#         current_loss += loss.item()

#         if i % 4 == 0 and i > 0:       # #  # #       #if i % 8 == 0 and i > 0:
#             # update the model using the optimizer
#             optimizer.step()
#             lr_scheduler.step()
#             # once we update the model we set the gradients to zero
#             optimizer.zero_grad()
#             # store the loss value for visualization
#             train_loss.append(current_loss/16)
#             total_loss=total_loss + current_loss/64
#             current_loss = 0


#         # update the model one last time for this epoch
#     optimizer.step()
#     lr_scheduler.step()
#     optimizer.zero_grad()
#     model.save_pretrained(directory)
#     print(directory + ' files saved')
#     print('total_loss',total_loss)


# #     validation###########################################################

#     model.eval().to(device)
#     valid_loss = []
#     valid_current_loss = 0
#     valid_total_loss=0

#     # iterate through each batch of t    he train data
#     for i, batch in enumerate(tqdm(valid_data)):

#         #print(batch)
#         batch = { k: v.to(device) for k, v in batch.items() }

#         #print(batch["labels"].shape)
#         #print(batch["input_ids"].shape)
#         # send 'input_ids', 'attention_mask' and 'labels' to the model
#         outputs = model(**batch)
#         # print(outputs)
#         loss=outputs.loss


#         valid_current_loss += loss.item()
#         #print(valid_current_loss)



#         valid_loss.append(valid_current_loss/batch_size)
#         valid_total_loss=valid_total_loss + valid_current_loss/batch_size
#         valid_current_loss = 0
#             #print("valid_total_loss ",valid_total_loss)

#     print("valid_total_loss ",valid_total_loss)
#     # valid_loss_epoch.append(valid_total_loss)
#     if valid_total_loss < valid_min_loss:
#         valid_min_loss = valid_total_loss
# #         model.save_pretrained(directory+"best_e_"+str(epoch))


#     if previous_total_valid_loss < valid_total_loss:
#         early_stoping_flag += 1
#     else:
#         early_stoping_flag=0
#     previous_total_valid_loss = valid_total_loss


#     if early_stoping_flag == early_stoping_patience:
#         print("early_stoping occure")
#         break



# #  target_names = ['O', 'B-PERSON', 'I-PERSON', 'B-LOCATION', 'I-LOCATION', 'B-ORGANIZATION', 'I-ORGANIZATION', 'B-FESTIVAL', 'I-FESTIVAL', 'B-GAME', 'I-GAME', 'B-LANGUAGE', 'I-LANGUAGE', 'B-LITERATURE', 'I-LITERATURE', 'B-MISC', 'I-MISC', 'B-NUMEX', 'I-NUMEX', 'B-RELIGION', 'I-RELIGION', 'B-TIMEX', 'I-TIMEX']

# # save vocabulary of the tokenizer
# #tokenizer.save_vocabulary(directory)
# # save the model weights and its configuration file

# print('All files saved')



training start
0
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_0
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_0 files saved
total_loss 0.765174181186012


100%|███████████████████████████████████████████| 24/24 [00:02<00:00, 10.60it/s]


valid_total_loss  0.020648444653488696
1
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_1
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_1 files saved
total_loss 0.24859311032196274


100%|███████████████████████████████████████████| 24/24 [00:02<00:00, 10.56it/s]


valid_total_loss  0.016335177060682327
2
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_2
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_2 files saved
total_loss 0.1621586470209877


...

valid_total_loss  0.021607325441436842
13
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_13
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_13 files saved
total_loss 0.09641751476920035


100%|███████████████████████████████████████████| 24/24 [00:02<00:00, 10.55it/s]


valid_total_loss  0.021607325441436842
14
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_14
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_14 files saved
total_loss 0.09641751476920035


100%|███████████████████████████████████████████| 24/24 [00:02<00:00, 10.53it/s]

valid_total_loss  0.021607325441436842
All files saved


In [ ]:
# best_model_path="/content/drive/MyDrive/textual_relatedness/models/epoch_3"#81.70
best_model_path="/content/drive/MyDrive/textual_relatedness/models/epoch_5"


num_labels = 1
model_max_length = max_len
batch_size= 16
min_batch_size=4
batch_size_c = batch_size / min_batch_size

In [ ]:

# Prediction
def  prediction(best_model_path, eval_data):


    predicted=[]
    tokanised_exam=[]
    true_labels=[]

    confusion = torch.zeros(num_labels, num_labels)
    predicted_for_classification_report=[]

    ##################################################
    #load best model


    print(best_model_path)

    model = RobertaForSequenceClassification.from_pretrained(best_model_path)



    model = model.eval().to(device)
    print("model device")
    # batch the train data so that each batch contains 4 examples (using 'batch_size')
    #test_data = torch.utils.data.DataLoader(test_dataset, batch_size=16)





    predicted=[]
    true_labels=[]


    predicted_for_classification_report=[]
    # iterate through each batch of t    he train data
    for i, batch in enumerate(tqdmn(eval_data)):


        with torch.no_grad():
#             print(batch)
            batch = { k: v.to(device) for k, v in batch.items() }
            # batch_labels = batch.pop("labels")
            outputs = model(**batch)

            logits = outputs.logits
            #print(logits.shape)



        # get the predicted values
        pred_values =logits.tolist()
        #print(pred_values)
        # true_labels += tags[idx][1:length-1].tolist()
        # true_labels_list.append(tags[idx][:length].tolist())

        #pred_values = pred_values.tolist()

        # print(pred_values)
        flattened_pred_values = [item for sublist in pred_values for item in sublist]

        print(flattened_pred_values)
        predicted +=flattened_pred_values

    # print(Counter(predicted))

    return predicted #here predicred is flat


In [ ]:
valid_predicted = prediction(best_model_path, valid_data)

/content/drive/MyDrive/textual_relatedness/models/epoch_4
model device


  0%|          | 0/69 [00:00<?, ?it/s]

[0.8677147030830383, 0.9685263633728027, 0.9671577215194702, 0.8463200330734253, 0.7330904006958008, 0.9127029180526733, 0.8296362161636353, 0.8470741510391235, 0.8778345584869385, 0.8357004523277283, 0.8777657747268677, 0.8939412236213684, 0.9307860732078552, 0.7076771855354309, 0.9769800305366516, 0.7721829414367676]
[0.8665282726287842, 0.6413089632987976, 0.8545132875442505, 0.8522595763206482, 0.8274969458580017, 0.715861976146698, 0.8624814748764038, 0.6947680711746216, 0.8666069507598877, 0.8458764553070068, 0.7459472417831421, 0.7477755546569824, 0.7157862782478333, 0.9091516733169556, 0.8417872786521912, 0.7588858008384705]
[0.8460123538970947, 0.9143412709236145, 0.7486365437507629, 0.8199021816253662, 0.9296796321868896, 0.9182645082473755, 0.8750166296958923, 0.736318826675415, 0.8660807609558105, 0.791685938835144, 0.8380727767944336, 0.5579389929771423, 0.7509849667549133, 0.8312090039253235, 0.6925581693649292, 0.8871711492538452]
[0.9125977158546448, 0.7389199733734131,

In [ ]:
test_predicted = prediction(best_model_path, test_data)

/content/drive/MyDrive/textual_relatedness/models/epoch_5
model device


  0%|          | 0/16 [00:00<?, ?it/s]

[0.6620236039161682, 0.7070541977882385, 0.270393967628479, 0.7107701897621155, 0.403751015663147, 0.2961830198764801, 0.7738701701164246, 0.7833516001701355, 0.6711634993553162, 0.6764703392982483, 0.8410447835922241, 0.8081191182136536, 0.6187582612037659, 0.29562583565711975, 0.2999381124973297, 0.7388707399368286]
[0.47775503993034363, 0.748953640460968, 0.7092359066009521, 0.6919556260108948, 0.31265878677368164, 0.4177931547164917, 0.2702993154525757, 0.20969220995903015, 0.23728050291538239, 0.8355153799057007, 0.4770984351634979, 0.49558475613594055, 0.29542794823646545, 0.5450635552406311, 0.3844943344593048, 0.3460504710674286]
[0.7247839570045471, 0.7747142910957336, 0.8487323522567749, 0.29098841547966003, 0.2962898015975952, 0.33116456866264343, 0.27865204215049744, 0.7781848311424255, 0.2883993089199066, 0.8006865382194519, 0.4051429331302643, 0.7066686153411865, 0.7844288349151611, 0.2641485035419464, 0.5790874361991882, 0.5622533559799194]
[0.5317178964614868, 0.4615609

In [ ]:
valid_predicted[0:10]

[0.8677147030830383,
 0.9685263633728027,
 0.9671577215194702,
 0.8463200330734253,
 0.7330904006958008,
 0.9127029180526733,
 0.8296362161636353,
 0.8470741510391235,
 0.8778345584869385,
 0.8357004523277283]

In [ ]:
test_predicted[:10]
print(len(test_predicted))
test_predicted
test_pdf["Pred_Score"] = test_predicted

250


In [ ]:
test_pdf

,PairID,Text,Split_Text,pred_scores,Pred_Score
0,ENG-dev-0000,The story is gripping and interesting.\nIt's a...,"[The story is gripping and interesting., It's ...",0.662024,0.662024
1,ENG-dev-0001,The majority of Southeast Alaska 's area is pa...,[The majority of Southeast Alaska 's area is p...,0.707054,0.707054
2,ENG-dev-0002,and from your post i think you are to young to...,[and from your post i think you are to young t...,0.270394,0.270394
3,ENG-dev-0003,The film 's success also made Dreamworks Anima...,[The film 's success also made Dreamworks Anim...,0.710770,0.710770
4,ENG-dev-0004,I am still confused about how I feel about thi...,[I am still confused about how I feel about th...,0.403751,0.403751
...,...,...,...,...,...
245,ENG-dev-0245,thats just how they are :( its a shame lol\nIt...,"[thats just how they are :( its a shame lol, I...",0.752000,0.752000
246,ENG-dev-0246,I feel sorry for the books that I will read af...,[I feel sorry for the books that I will read a...,0.300527,0.300527
247,ENG-dev-0247,Uwe Seeler -LRB- born 5 November 1936 in Hambu...,[Uwe Seeler -LRB- born 5 November 1936 in Hamb...,0.506162,0.506162
248,ENG-dev-0248,Waco is a city in and the county seat of McLen...,[Waco is a city in and the county seat of McLe...,0.486013,0.486013


In [ ]:
test_pdf[['PairID','Pred_Score']].to_csv("pred_eng_a.csv",index=False)

In [ ]:
# For testing
#label post allignment
def generate_score(predicted,true_labels):
  print("Pearson Correlation:", round(pearsonr(true_scores,pred_scores)[0]))

In [ ]:
generate_score( valid_predicted,valid_true_labels)

Pearson Correlation: 1


In [ ]:
for i,j in zip(valid_true_labels,valid_predicted):
  print(i,j)

1.0 0.8677147030830383
1.0 0.9685263633728027
1.0 0.9671577215194702
0.97 0.8463200330734253
0.97 0.7330904006958008
0.97 0.9127029180526733
0.94 0.8296362161636353
0.94 0.8470741510391235
0.94 0.8778345584869385
0.94 0.8357004523277283
0.94 0.8777657747268677
0.94 0.8939412236213684
0.94 0.9307860732078552
0.94 0.7076771855354309
0.94 0.9769800305366516
0.94 0.7721829414367676
0.94 0.8665282726287842
0.93 0.6413089632987976
0.91 0.8545132875442505
0.91 0.8522595763206482
0.91 0.8274969458580017
0.91 0.715861976146698
0.91 0.8624814748764038
0.91 0.6947680711746216
0.91 0.8666069507598877
0.91 0.8458764553070068
0.91 0.7459472417831421
0.91 0.7477755546569824
0.91 0.7157862782478333
0.91 0.9091516733169556
0.91 0.8417872786521912
0.91 0.7588858008384705
0.91 0.8460123538970947
0.91 0.9143412709236145
0.91 0.7486365437507629
0.91 0.8199021816253662
0.91 0.9296796321868896
0.91 0.9182645082473755
0.91 0.8750166296958923
0.91 0.736318826675415
0.91 0.8660807609558105
0.91 0.79168593883514